In [1]:
import pandas as pd

In [2]:
!pip install elasticsearch


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [7]:
#start clinidmap container

In [3]:
from __future__ import unicode_literals
from elasticsearch import Elasticsearch

import json

def get_elastic():
    return Elasticsearch(
        [{'host': 'localhost', 'port': 9200, 'scheme': 'http'}], 
        timeout=30, 
        max_retries=1, 
        retry_on_timeout=False
        )

# logger = logging.getLogger(__name__)
# logging.basicConfig(level=logging.INFO)

def query_search(q_dic, index_name, n_search=1000, elastic=None):
    '''
    q_dic: query dictionary in Elastic format ready to search 
    index_name: index in Elastic where to search
    '''
    if elastic is None:
        elastic = get_elastic()
    #body = json.dumps(q_dic, ensure_ascii=False)
    search_result = elastic.search(index=index_name, body=q_dic, size=n_search, from_=0)

    return search_result

def result2list_unique(result_dict, field):
    lst = []
    if result_dict['hits']['total']['value'] > 0:
        for hit in result_dict['hits']['hits']:
            if hit['_source'][field] not in lst: 
                lst.append(hit['_source'][field])
    return lst

def result2list_(result_dict, field):
    lst = []
    if result_dict['hits']['total']['value'] > 0:
        for hit in result_dict['hits']['hits']:
            lst.append(hit['_source'][field])
    return lst


def get_umls2sab_query(source_id, target_sab, lang):
    '''
    source_id: UMLS ID (CUI) to search
    target_sab: ontology mapped to this ID
    '''
    q_dic = {
        "query": { 
            "bool": { 
            "must": [
                {"match": {"CODE": source_id}},
                {"match": {"SAB": target_sab}}, 
                {"match": {"LAT": lang}}
                ]
                    }
                }
            }
    return q_dic

def query_and_search(code, index): 
    result_list = []
    q_dic = get_umls2sab_query(code, 'SCTSPA', 'SPA')
    result = query_search(q_dic, index)
    if result['hits']['total']['value'] > 0:
        for hit in result['hits']['hits']: 
            result_list.append(hit['_source'])

    return result_list

def get_umlstext_query(term, index, lang):
    '''
    source_id: UMLS ID (CUI) to search
    target_sab: ontology mapped to this ID
    '''
    q_dic = {
        "query": { 
            "bool": { 
            "must": [
                {"match": {
                  "STR": term
                }}, 
                {"match": {"LAT": lang}}
                ]
                    }
                }
            }
    return q_dic

def query_and_search_text(term, index): 
    result_list = []
    q_dic = get_umlstext_query(term, 'UMLS', 'ENG')
    result = query_search(q_dic, index)
    if result['hits']['total']['value'] > 0:
        for hit in result['hits']['hits']: 
            result_list.append(hit['_source'])

    return result_list

In [41]:
df_terms = pd.read_csv('st21pv_test_0928v1llama13b_entities_nofuzzy_linked_sap.tsv', sep='\t', header=0)

In [42]:
df_terms.head(10)

,document_id,sentence_id,doc_start_index,doc_end_index,sentence_entity_start,sentence_entity_end,text_segment,code
0,25847295,0,0,513,66,77,Nonylphenol,UMLS:C0068946
1,25847295,0,0,513,94,105,nonylphenol,UMLS:C0068946
2,25847295,0,0,513,370,381,nonylphenol,UMLS:C0068946
3,25847295,0,0,513,12,24,diethoxylate,UMLS:C0608491
4,25847295,0,0,513,382,394,diethoxylate,UMLS:C0608491
5,25847295,0,0,513,34,43,apoptosis,UMLS:C0162638
6,25847295,0,0,513,259,268,Apoptosis,UMLS:C0162638
7,25847295,0,0,513,437,446,apoptosis,UMLS:C0162638
8,25847295,0,0,513,55,59,PC12,UMLS:C0085262
9,25847295,0,0,513,496,500,PC12,UMLS:C0085262


In [43]:
df_terms[df_terms['text_segment'].isnull()]

,document_id,sentence_id,doc_start_index,doc_end_index,sentence_entity_start,sentence_entity_end,text_segment,code
14330,27884439,1,654,1061,319,321,NaN,UMLS:


In [44]:
df_terms.at[14330, 'text_segment'] = 'NA'

In [6]:
!pip install Levenshtein


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [7]:
import Levenshtein
Levenshtein.ratio('hello world', 'hello')

0.625

In [8]:
def select_best_candidate(text, results):
    max_ratio = 0
    max_candidate = results[0]['CUI']
    
    for result in results:
        ratio = Levenshtein.ratio(text, result['STR'])
        
        if ratio > max_ratio:
            max_ratio = ratio
            max_candidate = result['CUI']
            
    return max_candidate

In [19]:
for index, row in df_terms[14330:14333].iterrows():
    print(row['text_segment'])

NA
Dorpers
HTLTC


In [20]:
links = []
for index, row in df_terms[14330:].iterrows():
    results = query_and_search_text(row['text_segment'], 'umls')
    link = ''
    if len(results) > 0:        
        #link = 'UMLS:' + results[0]['CUI']
        #print(results)
        link = 'UMLS:' + select_best_candidate(row['text_segment'], results)
    
    links.append(link)

/tmp/ipykernel_243/124867022.py:7: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  return Elasticsearch(
/tmp/ipykernel_243/124867022.py:25: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  search_result = elastic.search(index=index_name, body=q_dic, size=n_search, from_=0)


In [21]:
len(links)

27447

In [22]:
links[0:10]

['UMLS:C0068946',
 'UMLS:C0068946',
 'UMLS:C0068946',
 '',
 '',
 'UMLS:C0162638',
 'UMLS:C0162638',
 'UMLS:C0162638',
 'UMLS:C0085262',
 'UMLS:C0085262']

In [23]:
query_and_search_text('apoptosis', 'umls')[0]

/tmp/ipykernel_243/124867022.py:7: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  return Elasticsearch(
/tmp/ipykernel_243/124867022.py:25: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  search_result = elastic.search(index=index_name, body=q_dic, size=n_search, from_=0)


{'CUI': 'C0162638',
 'LAT': 'ENG',
 'TS': 'P',
 'LUI': 'L0189749',
 'STT': 'PF',
 'SUI': 'S0257477',
 'ISPREF': 'N',
 'AUI': 'A0291520',
 'SAUI': 'None',
 'SCUI': 'None',
 'SDUI': '10059512',
 'SAB': 'MDR',
 'TTY': 'PT',
 'CODE': '10059512',
 'STR': 'Apoptosis',
 'SRL': '3',
 'SUPPRESS': 'N',
 'CVF': '256',
 '18': 'None'}

In [46]:
#df_terms['links'] = links
df_terms['links'] = df_terms['code']
df_terms.head()

,document_id,sentence_id,doc_start_index,doc_end_index,sentence_entity_start,sentence_entity_end,text_segment,code,links
0,25847295,0,0,513,66,77,Nonylphenol,UMLS:C0068946,UMLS:C0068946
1,25847295,0,0,513,94,105,nonylphenol,UMLS:C0068946,UMLS:C0068946
2,25847295,0,0,513,370,381,nonylphenol,UMLS:C0068946,UMLS:C0068946
3,25847295,0,0,513,12,24,diethoxylate,UMLS:C0608491,UMLS:C0608491
4,25847295,0,0,513,382,394,diethoxylate,UMLS:C0608491,UMLS:C0608491


In [47]:
df_test = df_terms

In [48]:
df_test['filename'] = ''
df_test['mark'] = 'T1'
df_test['label'] = 'ENFERMEDAD'
df_test['semantic_rel'] = 'EXACT'
df_test['off0'] = df_test['sentence_entity_start']
df_test['off1'] = df_test['sentence_entity_end']
df_test['span'] = df_test['text_segment']
df_test['code'] = df_test['links']

for index, row in df_test.iterrows():
    df_test.at[index, 'filename'] = str(row['document_id']) + '_' + str(row['sentence_id'])

In [49]:
df_test_new = df_test.drop(['document_id', 'sentence_id', 'doc_start_index', 'doc_end_index', 'sentence_entity_start', 'sentence_entity_end', 'text_segment', 'links'], axis=1)

In [50]:
df_test_new = df_test_new[['filename','mark','label','off0','off1',
                                           'span', 'code', 'semantic_rel']]

In [51]:
df_test_new[df_test_new['span'] == 'no-match'].index

Int64Index([], dtype='int64')

In [52]:
df_test_new = df_test_new.drop(df_test_new[df_test_new['span'] == 'no-match'].index)

In [53]:
df_test_new['add'] = False

In [54]:
df_test_new['sort_column'] = df_test_new['span'].str.len()

In [55]:
df_test_new = df_test_new.sort_values(by=['filename', 'sort_column'], ascending=False)

In [56]:
df_test_new.head()

,filename,mark,label,off0,off1,span,code,semantic_rel,add,sort_column
27445,28550165_2,T1,ENFERMEDAD,249,261,microtubules,UMLS:C0026046,EXACT,False,12
27446,28550165_2,T1,ENFERMEDAD,279,291,cytoskeleton,UMLS:C0010853,EXACT,False,12
27444,28550165_2,T1,ENFERMEDAD,60,67,tubulin,UMLS:C2937342,EXACT,False,7
27443,28550165_2,T1,ENFERMEDAD,39,45,CLASP2,UMLS:C1424727,EXACT,False,6
27442,28550165_2,T1,ENFERMEDAD,86,91,SOGA1,UMLS:C3469857,EXACT,False,5


In [57]:
df_test_groups = df_test_new.groupby(by=["filename"])

In [58]:
for group in df_test_groups:
    added_indices = []
    for index, row in group[1].iterrows():
        if row['off0'] not in added_indices and row['off1'] not in added_indices:
            added_indices.extend(range(row['off0'], row['off1']))
            df_test_new.at[index, 'add'] = True

In [59]:
df_test_new = df_test_new[df_test_new['add']]
df_test_new = df_test_new.sort_values(by=['filename', 'off0', 'off1'])
df_test_new = df_test_new[['filename','mark','label','off0','off1',
                                           'span', 'code', 'semantic_rel']]
df_test_new.head()

,filename,mark,label,off0,off1,span,code,semantic_rel
3,25847295_0,T1,ENFERMEDAD,12,24,diethoxylate,UMLS:C0608491,EXACT
5,25847295_0,T1,ENFERMEDAD,34,43,apoptosis,UMLS:C0162638,EXACT
8,25847295_0,T1,ENFERMEDAD,55,59,PC12,UMLS:C0085262,EXACT
0,25847295_0,T1,ENFERMEDAD,66,77,Nonylphenol,UMLS:C0068946,EXACT
1,25847295_0,T1,ENFERMEDAD,94,105,nonylphenol,UMLS:C0068946,EXACT


In [60]:
for index, row in df_test_new.iterrows():
    if row['span'].endswith('.') or row['span'].endswith(','):
        df_test_new.at[index, 'span'] = row['span'][0:-1]
        df_test_new.at[index, 'off1'] = row['off1']-1

In [61]:
#df_test_new = df_test_new.drop(['row'], axis=1)
df_test_new.head()

,filename,mark,label,off0,off1,span,code,semantic_rel
3,25847295_0,T1,ENFERMEDAD,12,24,diethoxylate,UMLS:C0608491,EXACT
5,25847295_0,T1,ENFERMEDAD,34,43,apoptosis,UMLS:C0162638,EXACT
8,25847295_0,T1,ENFERMEDAD,55,59,PC12,UMLS:C0085262,EXACT
0,25847295_0,T1,ENFERMEDAD,66,77,Nonylphenol,UMLS:C0068946,EXACT
1,25847295_0,T1,ENFERMEDAD,94,105,nonylphenol,UMLS:C0068946,EXACT


In [62]:
df_test_new.to_csv('vicuna_links/distemist_evaluation_library/toy-data/subtask-linking/st21pv_test_entities_llama_sap.tsv', sep='\t', index=False)

In [ ]:
df_test_new.shape

In [2]:
import pandas as pd
df_terms_gold = pd.read_csv('vicuna_links/distemist_evaluation_library/gs/subtask-linking/st21pv_test_entities.tsv', sep='\t', header=0)
df_terms_gold.head()

,filename,mark,label,off0,off1,span,code,semantic_rel
0,25847295_0,T1,ENFERMEDAD,34,43,apoptosis,UMLS:C0162638,EXACT
1,25847295_0,T1,ENFERMEDAD,55,65,PC12 cells,UMLS:C0085262,EXACT
2,25847295_0,T1,ENFERMEDAD,137,144,present,UMLS:C0150312,EXACT
3,25847295_0,T1,ENFERMEDAD,206,219,toxic effects,UMLS:C0600688,EXACT
4,25847295_0,T1,ENFERMEDAD,259,268,Apoptosis,UMLS:C0162638,EXACT


In [3]:
df_terms_gold = df_terms_gold[~df_terms_gold['span'].isnull()]

In [4]:
df_terms_gold.shape

(40097, 8)

In [ ]:
# EVAL linking approach ONLY

In [9]:
links = []
for index, row in df_terms_gold.iterrows():
    results = query_and_search_text(row['span'], 'umls')
    link = ''
    if len(results) > 0:        
        link = 'UMLS:' + results[0]['CUI']
    
    links.append(link)

/tmp/ipykernel_392/124867022.py:7: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  return Elasticsearch(
/tmp/ipykernel_392/124867022.py:25: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  search_result = elastic.search(index=index_name, body=q_dic, size=n_search, from_=0)


In [10]:
len(links)

40097

In [11]:
df_terms_gold['pred'] = links

In [15]:
df_terms_gold = df_terms_gold[['filename','mark','label','off0','off1',
                                           'span', 'pred', 'semantic_rel']]

In [16]:
df_terms_gold = df_terms_gold.rename(columns={"pred": "code"})

In [17]:
df_terms_gold.to_csv('vicuna_links/distemist_evaluation_library/gs/subtask-linking/st21pv_test_entities_predicted.tsv', sep='\t', index=False)

In [13]:
y_true = list(df_terms_gold['code'].values)
y_pred = list(df_terms_gold['pred'].values)

In [14]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred))

/home/sylvia/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sylvia/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sylvia/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sylvia/anaconda3/lib/python3.8/si

               precision    recall  f1-score   support

                    0.00      0.00      0.00         0
UMLS:C0000379       0.00      0.00      0.00         0
UMLS:C0000392       0.00      0.00      0.00         2
UMLS:C0000477       1.00      1.00      1.00         1
UMLS:C0000530       1.00      0.82      0.90        11
UMLS:C0000545       0.00      0.00      0.00         0
UMLS:C0000618       0.00      0.00      0.00         0
UMLS:C0000658       1.00      1.00      1.00         4
UMLS:C0000726       0.67      0.50      0.57         4
UMLS:C0000727       0.00      0.00      0.00         0
UMLS:C0000731       1.00      1.00      1.00         1
UMLS:C0000735       1.00      1.00      1.00         2
UMLS:C0000737       1.00      1.00      1.00         4
UMLS:C0000765       0.00      0.00      0.00         0
UMLS:C0000768       0.29      0.42      0.34        12
UMLS:C0000769       0.00      0.00      0.00         0
UMLS:C0000772       1.00      1.00      1.00         1
UMLS:C000

/home/sylvia/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sylvia/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
